In [4]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectFromModel

# Model Selection & Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, log_loss, confusion_matrix

# Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Classifier Algorithms
from sklearn.linear_model import LogisticRegression

# for vizzies
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import joblib
import requests
from urllib.parse import quote
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import scipy

In [8]:
# Import model
model = joblib.load("final_model.pkl")

# Import vectorizer
vectorizer = joblib.load("vectorizer.pkl")

/Users/tristantrechsel/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/tristantrechsel/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
def preprocess_tweet(tweet, tokenizer, stopwords_list):
    # Remove @mentions from tweet
    tweet = re.sub(r'@\w+', '', tweet)
    
    # Clean hashtags by removing the hash symbol and separating words by capital letters
    tweet = re.sub(r'#(\w+)', lambda x: ' '.join(re.findall(r'[A-Z]?[a-z]+', x.group(1))), tweet)
    
    # Remove URL's
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)
    
    # Standardize case (lowercase the text)
    tweet = tweet.lower()
    
    # Tokenize text using RegEx
    tokens = tokenizer.tokenize(tweet)
    
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stopwords_list]
    
    # Return the preprocessed text
    return (" ".join(filtered_tokens))

tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:'[a-z]+)?")
stopwords_list = stopwords.words('english')

In [11]:
def get_recent_tweets(query, bearer_token):
    url = "https://api.twitter.com/2/tweets/search/recent"
    encoded_query = quote(query)
    
    # Adding filters to exclude retweets, replies, and tweets with links
    params = {
        'query': f'{encoded_query} -is:retweet -is:reply -has:links lang:en',
        'max_results': '100',
    }
    headers = {
        'Authorization': f'Bearer {bearer_token}',
        'Content-Type': 'application/json',
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Request returned an error: {response.status_code} {response.text}")
    return response.json()

bearer_token = 'AAAAAAAAAAAAAAAAAAAAABYsrAEAAAAAvxcLhwG%2Fdf3Kc2TUwuAKsTMsykc%3DxmfUn4movLwukCRlbWafezA6xDDftHjaAcfWCtG5B4TUyAV2he'

In [12]:
# Call API for Beyonce tweets
topic = 'Beyonce'

tweets_json = get_recent_tweets(topic, bearer_token)

tweets_data = tweets_json.get('data', [])

beyonce_df = pd.DataFrame(tweets_data)

In [13]:
beyonce_df['tweet'] = beyonce_df.text.apply(lambda x: preprocess_tweet(x, tokenizer, stopwords_list))

In [14]:
beyonce_df.head()

,edit_history_tweet_ids,id,text,tweet
0,[1724965706620428509],1724965706620428509,Ummm why am I seeing Beyoncé and Michelle ship...,ummm seeing beyonc michelle ship edits
1,[1724965523878801641],1724965523878801641,People don’t understand how much if a cultural...,people understand much cultural phenomenon beyonc
2,[1724965314553692485],1724965314553692485,when moe don’t respond to my messages i be so ...,moe respond messages mad bitch beyonc
3,[1724964809479729155],1724964809479729155,just wanna meet someone that make me feel like...,wanna meet someone make feel like beyonce felt...
4,[1724964657847288297],1724964657847288297,15 days until Beyoncé ☺️,days beyonc


In [15]:
beyonce_X = beyonce_df.tweet
beyonce_X_vectorized = vectorizer.transform(beyonce_X)

TypeError: unsupported operand type(s) for *: 'csr_matrix' and 'csr_matrix'

In [65]:
model.predict(beyonce_X)

LogisticRegression(max_iter=1000)